In [4]:
from sklearn.cluster import KMeans

In [7]:
def train_cluster(nb, x, y): 
    X = x[y]

    tfidfvectorizer = TfidfVectorizer(max_features=8000,
                                     use_idf=True,
                                     tokenizer=nltk.word_tokenize,ngram_range=(1,3))

    tfidfmatrix = tfidfvectorizer.fit_transform(X)
    km= KMeans(nb)
    km.fit(tfidfmatrix)
    x['cluster']=km.labels_
    return x